In [1]:
!pip install transformers datasets


In [2]:
from google.colab import files
uploaded = files.upload()  # Upload your my_dataset.txt file


Saving my_dataset.txt to my_dataset.txt


In [3]:
with open("my_dataset.txt", "r", encoding="utf-8") as f:
    lines = f.read().split("\n\n")  # Separate entries by double newlines


In [4]:
from datasets import Dataset

# Read and split by double newline
with open("my_dataset.txt", "r", encoding="utf-8") as f:
    samples = f.read().split("\n\n")

# Convert to dataset format
data = [{"text": s.strip()} for s in samples if s.strip()]
dataset = Dataset.from_list(data)


In [5]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Prevents padding issues

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_steps=10,
    save_total_limit=1,
)


In [20]:
from transformers import GPT2LMHeadModel, Trainer, DataCollatorForLanguageModeling

# Ensure tokenizer has a pad_token (GPT-2 doesn't by default)
tokenizer.pad_token = tokenizer.eos_token  # ✅ Add this line if not already set

# Load the GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2", use_auth_token=False)

# Define the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")


/tmp/ipython-input-20-2709051577.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json',
 './gpt2-finetuned/tokenizer.json')

In [22]:
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

def get_response(prompt):
    input_text = f"Prompt: {prompt}\nResponse:"
    output = generator(
        input_text,
        max_length=100,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
    )[0]["generated_text"]

    if "Response:" in output:
        return output.split("Response:")[1].strip().split("\n")[0]
    return output
print(get_response("what is AI?"))


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


AI is the simulation of human intelligence processes by computers, and systems systems. AI is the ability of machines to perform tasks that typically require human intelligence.
